In [2]:
!pip install vaderSentiment

### VaderSentiment Python Package

You can read the orignal paper by authors [here](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf).

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import mlflow.pyfunc

In [5]:
# Define some input text

INPUT_TEXTS = [{'text': "This is a bad ass movie. You got to see it! :-)"},
               {'text': "Ricky Gervais is smart, witty, and creative!!!!!! :D"},
               {'text': "LOL, this guy fell off a chair while sleeping and snoring in a meeting"},
               {'text': "Men shoots himself while trying to steal a dog, OMG"},
               {'text': "Yay!! Another good phone interview. I nailed it!!"},
               {'text': "This is INSANE! I can't believe it. How could you do such a horrible thing?"}]

### Define a SocialMediaAnalyserModel

This is a subclass of [PythonModel](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.PythonModel)

In [6]:
class SocialMediaAnalyserModel(mlflow.pyfunc.PythonModel):

   def __init__(self):
      """
      Constructor for our Cusomized PyFunc Model Class
      """
      # Initialize an instance of vader analyser
      self._analyser = SentimentIntensityAnalyzer()

   def _score(self, text):
    """
    Private function to analyse the scores. It invokes model's 
    param: text to analyse
    return: sentiment analyses scores
    """
    scores = self._analyser.polarity_scores(text)
    return scores

   def predict(self, context, model_input):
    """
    Implement the predict function required for PythonModel
    """
    model_output = model_input.apply(lambda col: self._score(col))
    return model_output

In [7]:
def mlflow_run():
  
  # Save the conda environment for this model. 
  conda_env = {
    'channels': ['defaults', 'conda-forge'],
    'dependencies': [
        'python=3.7.6',
        'pip'],
    'pip': [
      'mlflow',
      'cloudpickle==1.3.0',
      'vaderSentiment==3.3.2'
    ],
    'name': 'mlflow-env'
  }
  
  # Model name and create an instance of PyFuncModel
  model_path = "vader"
  vader_model = SocialMediaAnalyserModel()
  with mlflow.start_run(run_name="Vader Sentiment Analysis") as run:
    # Log MLflow entities
    mlflow.pyfunc.log_model(model_path, python_model=vader_model, conda_env=conda_env)
    mlflow.log_param("algorithm", "VADER")
    mlflow.log_param("total_sentiments", len(INPUT_TEXTS))

    # Load back the model as a pyfunc_model for scoring
    model_uri = f"runs:/{run.info.run_uuid}/{model_path}"
    loaded_model = mlflow.pyfunc.load_model(model_uri)
    
    # Use predict function to score output from the model
    for i in range(len(INPUT_TEXTS)):
       text = INPUT_TEXTS[i]['text']
       mlflow.log_param(f"text_{i}", text)
      
      # Use predict function to score output from the model
       model_input = pd.DataFrame([text])
       scores = loaded_model.predict(model_input)
       print(f"<{text}> ------- {str(scores[0])}>")
       for index, value in scores.items():
          [mlflow.log_metric(f"{key}_{i}", value) for key, value in value.items()]
          
    return run.info.run_id

Log parameters, metrics, and model with MLflow APIs

In [8]:
 mlflow_run()

<This is a bad ass movie. You got to see it! :-)> ------- {'neg': 0.0, 'neu': 0.542, 'pos': 0.458, 'compound': 0.7644}>
<Ricky Gervais is smart, witty, and creative!!!!!! :D> ------- {'neg': 0.0, 'neu': 0.316, 'pos': 0.684, 'compound': 0.8957}>
<LOL, this guy fell off a chair while sleeping and snoring in a meeting> ------- {'neg': 0.0, 'neu': 0.786, 'pos': 0.214, 'compound': 0.5473}>
<Men shoots himself while trying to steal a dog, OMG> ------- {'neg': 0.262, 'neu': 0.738, 'pos': 0.0, 'compound': -0.4939}>
<Yay!! Another good phone interview. I nailed it!!> ------- {'neg': 0.0, 'neu': 0.446, 'pos': 0.554, 'compound': 0.816}>
<This is INSANE! I can't believe it. How could you do such a horrible thing?> ------- {'neg': 0.357, 'neu': 0.643, 'pos': 0.0, 'compound': -0.8034}>


'ef7d7ef8d30745c7a9f3a4ca8c5efaca'

#### Excercise Assignment

 * Create some text input
 * load the model as a PyFuncModel
 * Invoke its predict method with each text
 * Do the scores change if you remove emojis and chat speak" or "text speak?
 * Print the scores